# Python에서 벡터 검색 (Azure AI Search)

이 코드는 Azure AI Search의 push API를 사용하여 벡터를 검색 인덱스에 삽입하는 방법을 보여줍니다:

+ 인덱스 스키마 생성
+ 로컬 폴더에서 샘플 데이터 로드
+ Azure OpenAI의 text-embedding-3-large 모델을 사용하여 문서 임베딩 생성
+ 벡터 및 비벡터 필드를 Azure AI Search에 인덱싱
+ 메타데이터 필터링 및 하이브리드(텍스트 + 벡터) 검색을 포함한 다양한 벡터 및 하이브리드 쿼리 실행

코드는 Azure OpenAI를 사용하여 title과 content 필드의 임베딩을 생성합니다. 이 데모를 실행하려면 Azure OpenAI에 대한 액세스가 필요합니다.

코드는 입력 데이터가 포함된 `text-sample.json` 파일을 읽어 임베딩을 생성합니다.

출력은 사람이 읽을 수 있는 텍스트와 검색 인덱스에 삽입할 수 있는 임베딩의 조합입니다.

## 사전 준비 사항

+ [Azure OpenAI에 대한 액세스](https://aka.ms/oai/access)가 있는 Azure 구독. Azure OpenAI 서비스 이름과 API 키가 필요합니다.

+ text-embedding-3-large 임베딩 모델의 배포.

+ Azure AI Search(모든 티어 가능). 벡터 인덱스를 위해 충분한 용량이 있는 서비스를 선택하세요. 벡터 인덱스에는 Basic 이상을 권장합니다. 하이브리드 쿼리에서 시맨틱 랭킹을 사용하려면 [시맨틱 랭킹 활성화](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable)가 필요합니다.

이 예제는 Python 3.11, [Python 확장 기능이 포함된 Visual Studio Code](https://code.visualstudio.com/docs/python/python-tutorial), 그리고 [Jupyter 확장 기능](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)에서 테스트되었습니다.

### Visual Studio Code에서 Python 가상 환경 설정

1. 명령 팔레트(Ctrl+Shift+P)를 엽니다.
1. **Python: Create Environment**를 검색합니다.
1. **Venv**를 선택합니다.
1. Python 인터프리터를 선택합니다. 3.10 이상을 선택하세요.

설정에는 약간의 시간이 걸릴 수 있습니다. 문제가 발생하면 [VS Code의 Python 환경](https://code.visualstudio.com/docs/python/environments) 문서를 참고하세요.

## 필수 라이브러리 및 환경 변수 불러오기

In [5]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True, dotenv_path="../.env") 


endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "vectest")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY", "") if len(os.getenv("AZURE_OPENAI_KEY", "")) > 0 else None
azure_openai_embedding_deployment = os.getenv("EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 1536))
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")

## 임베딩 생성
데이터를 읽고, OpenAI 임베딩을 생성하여 Azure AI Search 인덱스에 삽입할 수 있는 형식으로 내보냅니다:

In [2]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_deployment=azure_openai_embedding_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)

# Generate Document Embeddings using OpenAI 3 large
# Read the text-sample.json
path = os.path.join('..', 'data', 'text-sample.json')
with open(path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

titles = [item['title'] for item in input_data]
content = [item['content'] for item in input_data]
title_response = client.embeddings.create(input=titles, model=embedding_model_name)
title_embeddings = [item.embedding for item in title_response.data]
content_response = client.embeddings.create(input=content, model=embedding_model_name)
content_embeddings = [item.embedding for item in content_response.data]

# Generate embeddings for title and content fields
for i, item in enumerate(input_data):
    title = item['title']
    content = item['content']
    item['titleVector'] = title_embeddings[i]
    item['contentVector'] = content_embeddings[i]

# Output embeddings to docVectors.json file
output_path = os.path.join('output', 'docVectors.json')
output_directory = os.path.dirname(output_path)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, "w") as f:
    json.dump(input_data, f)

## 검색 인덱스 생성

검색 인덱스 스키마와 벡터 검색 구성을 만듭니다. 오류가 발생하면 검색 서비스의 사용 가능한 할당량을 확인하고 .env 파일에서 고유한 인덱스 이름을 사용하고 있는지 확인하세요.

In [7]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)


# Create a search index
index_client = SearchIndexClient(
    endpoint=endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=azure_openai_embedding_dimensions, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=azure_openai_embedding_dimensions, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myVectorizer"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myVectorizer",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_embedding_deployment,
                model_name=embedding_model_name,
                api_key=azure_openai_key
            )
        )
    ]
)



semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f'{result.name} created')


vectest created


## 텍스트와 임베딩을 벡터 스토어에 삽입
JSON 데이터의 텍스트와 메타데이터를 벡터 스토어에 추가합니다:

In [8]:
from azure.search.documents import SearchClient
import json

# Upload some documents to the index
output_path = os.path.join('output', 'docVectors.json')
output_directory = os.path.dirname(output_path)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(output_path, 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 108 documents


대량의 문서를 인덱싱하는 경우, `SearchIndexingBufferedSender`를 사용하면 자동으로 배치 처리를 하여 최적화된 방식으로 문서를 인덱싱할 수 있습니다:

In [ ]:
from azure.search.documents import SearchIndexingBufferedSender

# Upload some documents to the index  
with open(output_path, 'r') as file:  
    documents = json.load(file)  
  
# Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
with SearchIndexingBufferedSender(  
    endpoint=endpoint,  
    index_name=index_name,  
    credential=credential,  
) as batch_client:  
    # Add upload actions for all documents  
    batch_client.upload_documents(documents=documents)  
print(f"Uploaded {len(documents)} documents in total")  


Uploaded 108 documents in total


In [9]:
# Helper code to print results

from azure.search.documents import SearchItemPaged

def print_results(results: SearchItemPaged[dict]):
    semantic_answers = results.get_answers()
    if semantic_answers:
        for answer in semantic_answers:
            if answer.highlights:
                print(f"Semantic Answer: {answer.highlights}")
            else:
                print(f"Semantic Answer: {answer.text}")
            print(f"Semantic Answer Score: {answer.score}\n")

    for result in results:
        print(f"Title: {result['title']}")  
        print(f"Score: {result['@search.score']}")
        if result.get('@search.reranker_score'):
            print(f"Reranker Score: {result['@search.reranker_score']}")
        print(f"Content: {result['content']}")  
        print(f"Category: {result['category']}\n")

        captions = result["@search.captions"]
        if captions:
            caption = captions[0]
            if caption.highlights:
                print(f"Caption: {caption.highlights}\n")
            else:
                print(f"Caption: {caption.text}\n")

## 벡터 유사도 검색 수행

이 예제는 미리 계산된 임베딩을 사용한 순수 벡터 검색을 보여줍니다.

In [10]:
from azure.search.documents.models import VectorizedQuery

# Pure Vector Search
query = "소프트웨어 개발도구들"  
embedding = client.embeddings.create(input=query, model=embedding_model_name).data[0].embedding

# 50 is an optimal value for k_nearest_neighbors when performing vector search
# To learn more about how vector ranking works, please visit https://learn.microsoft.com/azure/search/vector-search-ranking
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=50, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)

Title: Azure 데브옵스
Score: 0.6362665
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 데브테스트 랩
Score: 0.6100094
Content: Azure 데브테스트 랩은 Azure에서 개발 및 테스트 환경을 생성, 관리 및 공유할 수 있는 완전 관리형 서비스입니다. 사용자 지정 템플릿, 비용 관리, Azure DevOps와의 통합 등의 기능을 제공합니다. Windows, Linux, Kubernetes 등 다양한 플랫폼을 지원합니다. Azure 데브테스트 랩을 사용하여 애플리케이션 개발 수명 주기를 개선하고, 비용을 절감하며, 환경의 일관성을 보장할 수 있습니다. Azure Virtual Machines, Azure App Service 등 다른 Azure 서비스와 통합됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.6005465
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bitbucket 등 다른 Azure 서비스와의 통합도 제공합니

## 벡터화 가능한 텍스트 쿼리를 사용한 벡터 유사도 검색 수행

이 예제는 벡터화 가능한 텍스트 쿼리를 사용한 순수 벡터 검색을 보여줍니다. 텍스트만 전달하면 벡터라이저가 쿼리 벡터화를 처리합니다.

In [11]:
from azure.search.documents.models import VectorizableTextQuery

# Pure Vector Search
query = "소프트웨어 개발도구들"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)

Title: Azure 데브옵스
Score: 0.6362665
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 데브테스트 랩
Score: 0.6100094
Content: Azure 데브테스트 랩은 Azure에서 개발 및 테스트 환경을 생성, 관리 및 공유할 수 있는 완전 관리형 서비스입니다. 사용자 지정 템플릿, 비용 관리, Azure DevOps와의 통합 등의 기능을 제공합니다. Windows, Linux, Kubernetes 등 다양한 플랫폼을 지원합니다. Azure 데브테스트 랩을 사용하여 애플리케이션 개발 수명 주기를 개선하고, 비용을 절감하며, 환경의 일관성을 보장할 수 있습니다. Azure Virtual Machines, Azure App Service 등 다른 Azure 서비스와 통합됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.6005465
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bitbucket 등 다른 Azure 서비스와의 통합도 제공합니

이 예제는 Azure OpenAI의 text-embedding-3-large 다국어 기능을 보여주는 순수 벡터 검색 예시입니다.

In [22]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "tools voor softwareontwikkeling"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)



Title: Azure 데브옵스
Score: 0.5774976
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 데브테스트 랩
Score: 0.5612859
Content: Azure 데브테스트 랩은 Azure에서 개발 및 테스트 환경을 생성, 관리 및 공유할 수 있는 완전 관리형 서비스입니다. 사용자 지정 템플릿, 비용 관리, Azure DevOps와의 통합 등의 기능을 제공합니다. Windows, Linux, Kubernetes 등 다양한 플랫폼을 지원합니다. Azure 데브테스트 랩을 사용하여 애플리케이션 개발 수명 주기를 개선하고, 비용을 절감하며, 환경의 일관성을 보장할 수 있습니다. Azure Virtual Machines, Azure App Service 등 다른 Azure 서비스와 통합됩니다.
Category: 개발자 도구

Title: Azure 마이그레이트
Score: 0.54510754
Content: Azure 마이그레이트는 온프레미스 워크로드를 Azure로 마이그레이션할 수 있는 클라우드 마이그레이션 서비스입니다. VM 호환성 평가, 성능 기반 크기 조정, 비용 추정 등의 기능을 제공합니다. Windows, Linux, VMware 등 다양한 플랫폼을 지원합니다. Azure 마이그레이트를 사용하여 마이그레이션을 계획하고, 워크로드의 호환성을 보장하며, 비용을 최적화할 수 있습니다. Azure Site Reco

## Exhaustive KNN(정확한 최근접 이웃) 검색 수행

이 예제는 인덱스 종류(HNSW 또는 ExhaustiveKNN)에 상관없이 벡터 인덱스를 전체 탐색하는 방법을 보여줍니다. 이를 통해 ground-truth 값을 계산할 수 있습니다.

In [13]:
# Pure Vector Search
query = "소프트웨어 개발도구들"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)

Title: Azure 데브옵스
Score: 0.6362665
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 데브테스트 랩
Score: 0.6100094
Content: Azure 데브테스트 랩은 Azure에서 개발 및 테스트 환경을 생성, 관리 및 공유할 수 있는 완전 관리형 서비스입니다. 사용자 지정 템플릿, 비용 관리, Azure DevOps와의 통합 등의 기능을 제공합니다. Windows, Linux, Kubernetes 등 다양한 플랫폼을 지원합니다. Azure 데브테스트 랩을 사용하여 애플리케이션 개발 수명 주기를 개선하고, 비용을 절감하며, 환경의 일관성을 보장할 수 있습니다. Azure Virtual Machines, Azure App Service 등 다른 Azure 서비스와 통합됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.6005465
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bitbucket 등 다른 Azure 서비스와의 통합도 제공합니

## 교차 필드 벡터 검색 수행

이 예제는 여러 벡터 필드를 동시에 쿼리할 수 있는 교차 필드 벡터 검색을 보여줍니다. 쿼리할 벡터 필드에 동일한 임베딩 모델을 사용했는지 확인하세요.

In [14]:
# Pure Vector Search
query = "소프트웨어 개발도구들"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector, titleVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)


Title: Azure 데브옵스
Score: 0.03253968432545662
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.0317540317773819
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bitbucket 등 다른 Azure 서비스와의 통합도 제공합니다.
Category: 웹

Title: Azure 애플리케이션 인사이트
Score: 0.03125763311982155
Content: Azure 애플리케이션 인사이트는 애플리케이션 및 인프라를 모니터링, 진단 및 문제 해결할 수 있는 애플리케이션 성능 관리 서비스입니다. 실시간 원격 분석, 종속성 매핑, 실시간 메트릭 등의 기능을 제공합니다. .NET, Java, Node.js, Python 등 다양한 플랫폼을 지원합니다. Azure 애플리케이션 인사이트를 사용하여 문제를 감지하고 진단하며, 성능을 최적화하고, 애플리케이션의 가용성을 보장할 수 있습니다. Azure Monitor, Azure Log Analytics

## 다중 벡터 검색 수행

이 예제는 여러 쿼리 벡터를 전달하여 여러 벡터 필드를 동시에 쿼리할 수 있는 교차 필드 벡터 검색을 보여줍니다. 이 경우, 서로 다른 임베딩 모델의 쿼리 벡터를 인덱스의 해당 벡터 필드에 전달할 수 있습니다.

In [15]:
# Multi-Vector Search
query = "소프트웨어 개발도구들"  
  

vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="titleVector")
vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)


Title: Azure 데브옵스
Score: 0.03253968432545662
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.0317540317773819
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bitbucket 등 다른 Azure 서비스와의 통합도 제공합니다.
Category: 웹

Title: Azure 애플리케이션 인사이트
Score: 0.03125763311982155
Content: Azure 애플리케이션 인사이트는 애플리케이션 및 인프라를 모니터링, 진단 및 문제 해결할 수 있는 애플리케이션 성능 관리 서비스입니다. 실시간 원격 분석, 종속성 매핑, 실시간 메트릭 등의 기능을 제공합니다. .NET, Java, Node.js, Python 등 다양한 플랫폼을 지원합니다. Azure 애플리케이션 인사이트를 사용하여 문제를 감지하고 진단하며, 성능을 최적화하고, 애플리케이션의 가용성을 보장할 수 있습니다. Azure Monitor, Azure Log Analytics

## 가중치가 적용된 다중 벡터 검색 수행

이 예제는 여러 쿼리 벡터에 서로 다른 가중치를 적용하여 여러 벡터 필드를 동시에 쿼리할 수 있는 교차 필드 벡터 검색을 보여줍니다.

In [16]:
# Multi-Vector Search
query = "소프트웨어 개발도구들"  
  

vector_query_1 = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="titleVector", weight=2)
vector_query_2 = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector", weight=0.5)

results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_2],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)

Title: Azure 데이터 익스플로러
Score: 0.04047619178891182
Content: Azure 데이터 익스플로러는 대량의 데이터를 실시간으로 분석할 수 있는 빠르고 완전 관리형 데이터 분석 서비스입니다. 데이터 수집, 쿼리, 시각화 등의 기능을 제공합니다. Azure Event Hubs, Azure IoT Hub, Azure Blob Storage 등 다양한 데이터 소스를 지원합니다. 데이터 익스플로러를 사용하여 로그를 분석하고, 애플리케이션을 모니터링하며, 데이터에 대한 인사이트를 얻을 수 있습니다. Azure Synapse Analytics, Azure Machine Learning 등 다른 Azure 서비스와 통합됩니다.
Category: 분석

Title: Azure 데브옵스
Score: 0.04007936641573906
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 앱 서비스
Score: 0.03931451588869095
Content: Azure 앱 서비스는 웹 앱을 구축, 배포 및 확장할 수 있는 완전 관리형 플랫폼입니다. 웹 앱, 모바일 앱 백엔드, RESTful API를 호스팅할 수 있습니다. .NET, Java, Node.js, Python, PHP 등 다양한 언어와 프레임워크를 지원합니다. 자동 확장 및 부하 분산 기능이 내장되어 있으며, Azure DevOps, GitHub, Bit

## 필터가 적용된 순수 벡터 검색 수행
이 예제는 인덱스에 필터를 적용하는 방법을 보여줍니다. Pre-Filtering(기본값) 또는 Post-Filtering 중 원하는 방식을 선택할 수 있습니다.

In [17]:
from azure.search.documents.models import VectorFilterMode

# Pure Vector Search
query = "소프트웨어 개발도구들"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector")

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq '개발자 도구'",
    select=["title", "content", "category"],
    top=3
)
  
print_results(results)


Title: Azure 데브옵스
Score: 0.6362665
Content: Azure 데브옵스는 애플리케이션 계획, 빌드, 배포를 지원하는 서비스 모음입니다. Azure Boards(작업 추적), Azure Repos(소스 코드 관리), Azure Pipelines(CI/CD), Azure Test Plans(테스트), Azure Artifacts(패키지 관리) 등을 포함합니다. 다양한 언어, 프레임워크, 플랫폼을 지원하며, 기존 개발 도구 및 프로세스와 쉽게 통합할 수 있습니다. Azure App Service, Azure Functions 등 다른 Azure 서비스와도 연동됩니다.
Category: 개발자 도구

Title: Azure 데브테스트 랩
Score: 0.6100094
Content: Azure 데브테스트 랩은 Azure에서 개발 및 테스트 환경을 생성, 관리 및 공유할 수 있는 완전 관리형 서비스입니다. 사용자 지정 템플릿, 비용 관리, Azure DevOps와의 통합 등의 기능을 제공합니다. Windows, Linux, Kubernetes 등 다양한 플랫폼을 지원합니다. Azure 데브테스트 랩을 사용하여 애플리케이션 개발 수명 주기를 개선하고, 비용을 절감하며, 환경의 일관성을 보장할 수 있습니다. Azure Virtual Machines, Azure App Service 등 다른 Azure 서비스와 통합됩니다.
Category: 개발자 도구

Title: Azure 앱 구성
Score: 0.5799921
Content: Azure 앱 구성은 애플리케이션 설정 및 기능 플래그를 중앙에서 관리하고 배포할 수 있는 완전 관리형 구성 서비스입니다. 키-값 저장소, 버전 관리, 액세스 제어 등의 기능을 제공합니다. .NET, Java, Node.js, Python 등 다양한 플랫폼을 지원합니다. Azure 앱 구성을 사용하여 애플리케이션을 구축 및 배포하고, 설정의 일관성을 보장하며, 애플리케이션 수명 주기를 개선할 수 있습니다. Azure A

## 하이브리드 검색 수행

In [18]:
# Hybrid Search
query = "대용량 저장하려면 어떤 솔루션이 좋을까?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)


Title: Azure 스토리지
Score: 0.01666666753590107
Content: Azure 스토리지는 다양한 데이터 유형(Blob, 파일, 큐, 테이블)을 지원하는 확장 가능하고 내구성 높은 클라우드 스토리지 서비스입니다. 비정형 데이터에 대한 대규모 객체 저장소를 제공합니다. 데이터 중복 및 지리적 복제를 지원하여 높은 내구성과 가용성을 보장합니다. REST API, SDK, Azure Portal 등 다양한 데이터 접근 및 관리 옵션을 제공하며, 저장 및 전송 시 암호화를 통해 데이터를 보호할 수 있습니다.
Category: 스토리지

Title: Azure 블롭 스토리지
Score: 0.016393441706895828
Content: Azure 블롭 스토리지는 비정형 데이터를 위한 확장 가능하고 내구성 있으며 고성능의 객체 스토리지 서비스입니다. 데이터 중복, 지리적 복제, 세분화된 액세스 제어 등의 기능을 제공합니다. 이미지, 문서, 비디오 등 다양한 데이터 유형을 지원합니다. 블롭 스토리지를 사용하여 데이터를 저장 및 관리하고, 데이터 레이크를 구축하며, 빅데이터 분석 솔루션을 개발할 수 있습니다. Azure CDN, Azure Functions, Azure Machine Learning 등 다른 Azure 서비스와 통합됩니다.
Category: 스토리지

Title: Azure 데이터 레이크 스토리지
Score: 0.016129031777381897
Content: Azure 데이터 레이크 스토리지는 빅데이터 분석을 위한 확장 가능하고 안전하며 비용 효율적인 스토리지 서비스입니다. 무제한 스토리지, 계층적 네임스페이스, 세분화된 액세스 제어 등의 기능을 제공합니다. 구조화된 데이터, 반구조화된 데이터, 비정형 데이터 등 다양한 데이터 유형을 지원합니다. 데이터 레이크 스토리지를 사용하여 데이터를 저장 및 분석하고, 데이터 레이크를 구축하며, 다른 Azure 서비스와 통합할 수 있습니다. Azure Databricks, Azure M

## 가중치가 적용된 하이브리드 검색 수행

벡터 쿼리와 텍스트 쿼리의 가중치를 조정합니다.

In [19]:
# Hybrid Search
query = "대용량 저장하려면 어떤 솔루션이 좋을까?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector", weight=0.2)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
print_results(results)

Title: Azure 스토리지
Score: 0.0033333336468786
Content: Azure 스토리지는 다양한 데이터 유형(Blob, 파일, 큐, 테이블)을 지원하는 확장 가능하고 내구성 높은 클라우드 스토리지 서비스입니다. 비정형 데이터에 대한 대규모 객체 저장소를 제공합니다. 데이터 중복 및 지리적 복제를 지원하여 높은 내구성과 가용성을 보장합니다. REST API, SDK, Azure Portal 등 다양한 데이터 접근 및 관리 옵션을 제공하며, 저장 및 전송 시 암호화를 통해 데이터를 보호할 수 있습니다.
Category: 스토리지

Title: Azure 블롭 스토리지
Score: 0.003278688294813037
Content: Azure 블롭 스토리지는 비정형 데이터를 위한 확장 가능하고 내구성 있으며 고성능의 객체 스토리지 서비스입니다. 데이터 중복, 지리적 복제, 세분화된 액세스 제어 등의 기능을 제공합니다. 이미지, 문서, 비디오 등 다양한 데이터 유형을 지원합니다. 블롭 스토리지를 사용하여 데이터를 저장 및 관리하고, 데이터 레이크를 구축하며, 빅데이터 분석 솔루션을 개발할 수 있습니다. Azure CDN, Azure Functions, Azure Machine Learning 등 다른 Azure 서비스와 통합됩니다.
Category: 스토리지

Title: Azure 데이터 레이크 스토리지
Score: 0.003225806402042508
Content: Azure 데이터 레이크 스토리지는 빅데이터 분석을 위한 확장 가능하고 안전하며 비용 효율적인 스토리지 서비스입니다. 무제한 스토리지, 계층적 네임스페이스, 세분화된 액세스 제어 등의 기능을 제공합니다. 구조화된 데이터, 반구조화된 데이터, 비정형 데이터 등 다양한 데이터 유형을 지원합니다. 데이터 레이크 스토리지를 사용하여 데이터를 저장 및 분석하고, 데이터 레이크를 구축하며, 다른 Azure 서비스와 통합할 수 있습니다. Azure Databricks, Azure Ma

## 시맨틱 하이브리드 검색 수행

In [20]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "azure 탐색?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector", exhaustive=True)

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

print_results(results)


Semantic Answer: Azure 퀀텀은 Azure를 사용하여 양자 애플리케이션을 구축하고 실행할 수 있는 완전 관리형 양자 컴퓨팅 서비스입니다. 양자 하드웨어, 양자 시뮬레이터, 양자 최적화 알고리즘 등의 기능을 제공합니다. Q#, Python, C# 등 다양한 프로그래밍 언어를 지원합니다. <em>Azure 퀀텀을</em><em> 사용하여 양자 컴퓨팅을 탐색하</em>고, 양자 알고리즘을 개발하며, 복잡한 최적화 문제를 해결할 수 있습니다. Azure Machine Learning, Azure Functions 등 다른 Azure 서비스와 통합됩니다.
Semantic Answer Score: 0.9900000095367432

Title: Azure 코그니티브 검색
Score: 0.03229166567325592
Reranker Score: 2.621738910675049
Content: Azure 코그니티브 검색은 애플리케이션에 풍부한 검색 기능을 구축할 수 있는 완전 관리형 검색 서비스입니다. 전체 텍스트 검색, 계층형 탐색, 필터 등의 기능을 제공합니다. Azure SQL Database, Azure Blob Storage, Azure Cosmos DB 등 다양한 데이터 소스를 지원합니다. 코그니티브 검색을 사용하여 데이터를 인덱싱하고, 사용자 정의 점수 프로필을 생성하며, 다른 Azure 서비스와 통합할 수 있습니다. Azure Cognitive Services, Azure Machine Learning 등 다른 Azure 서비스와 통합됩니다.
Category: AI + 머신러닝

Caption: <em>Azure 코그니티브 검색은</em> 애플리케이션에 풍부한 검색 기능을 구축할 수 있는 완전 관리형<em> 검색 </em>서비스입니다. <em>전체 텍스트 검색, 계층형 탐색, 필터 등의 기능을 </em>제공합니다. <em>Azure SQL Database, Azure Blob Storage, Azure Cosmos DB 등 다양한 데이터 소

## 쿼리 리라이팅이 적용된 시맨틱 하이브리드 검색 수행

+ [쿼리 리라이팅](https://learn.microsoft.com/azure/search/semantic-how-to-query-rewrite)과 시맨틱 랭커를 함께 사용합니다.
+ debug 파라미터를 queryRewrites로 설정하면 쿼리가 어떻게 리라이팅되었는지 확인할 수 있습니다.
+ 프로덕션 환경에서는 debug 파라미터 사용을 피하세요.

In [21]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType, QueryDebugMode
from typing import Optional

# Workaround required to use debug query rewrites with the preview SDK
import azure.search.documents._generated.models
azure.search.documents._generated.models.SearchDocumentsResult._attribute_map["debug_info"]["key"] = "@search\\.debug"
from azure.search.documents._generated.models import DebugInfo
import azure.search.documents._paging
def get_debug_info(self) -> Optional[DebugInfo]:
    self.continuation_token = None
    return self._response.debug_info
azure.search.documents._paging.SearchPageIterator.get_debug_info = azure.search.documents._paging._ensure_response(get_debug_info)
azure.search.documents._paging.SearchItemPaged.get_debug_info = lambda self: self._first_iterator_instance().get_debug_info()

# Semantic Hybrid Search with Query Rewriting
query = "Azure 검색"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="contentVector", query_rewrites="generative|count-5")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_rewrites="generative|count-5",
    query_language="en",
    debug=QueryDebugMode.QUERY_REWRITES,
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

text_query_rewrites = results.get_debug_info().query_rewrites.text.rewrites
vector_query_rewrites = results.get_debug_info().query_rewrites.vectors[0].rewrites
print_results(results)

print("Text Query Rewrites:")
print(text_query_rewrites)
print("Vector Query Rewrites:")
print(vector_query_rewrites)

Semantic Answer: Azure 코그니티브 서비스는 사전 구축된 모델 및 알고리즘을 사용하여 지능형 애플리케이션을 구축할 수 있는 AI 서비스 및 API 모음입니다. 컴퓨터 비전, 음성 인식, 자연어 처리 등의 기능을 제공합니다.. NET, Java, Node.js, Python 등 다양한 플랫폼을 지원합니다. Azure 코그니티브 서비스를 사용하여 챗봇을 구축하고, 이미지 및 비디오를 분석하며, 텍스트를 처리 및 이해할 수 있습니다. <em>Azure Machine Learning, Azure Cognitive Search </em>등 다른<em> Azure </em>서비스와 통합됩니다.
Semantic Answer Score: 0.9919999837875366

Title: Azure 코그니티브 검색
Score: 0.03333333507180214
Reranker Score: 3.147062063217163
Content: Azure 코그니티브 검색은 애플리케이션에 풍부한 검색 기능을 구축할 수 있는 완전 관리형 검색 서비스입니다. 전체 텍스트 검색, 계층형 탐색, 필터 등의 기능을 제공합니다. Azure SQL Database, Azure Blob Storage, Azure Cosmos DB 등 다양한 데이터 소스를 지원합니다. 코그니티브 검색을 사용하여 데이터를 인덱싱하고, 사용자 정의 점수 프로필을 생성하며, 다른 Azure 서비스와 통합할 수 있습니다. Azure Cognitive Services, Azure Machine Learning 등 다른 Azure 서비스와 통합됩니다.
Category: AI + 머신러닝

Caption: <em>Azure 코그니티브 </em>검색은<em> 애플리케이션에 풍부한 검색 기능을 구축할 </em>수 있는<em> 완전 관리형 검색 서비스입</em>니다. <em>전체 텍스트 검색, 계층형 탐색, 필터 등의 기능을 제공합니다.</em> <em>Azure SQL Database, Azure Blob Storag